In [0]:
spark.sql("USE CATALOG hive_metastore")
storage_account = "databricks180820251"

In [0]:
from delta.tables import *
from pyspark.sql.functions import lit,col


In [0]:
orders = DeltaTable.forPath(spark, f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/orders")

In [0]:
type(orders)

In [0]:
orders = DeltaTable.forName(spark,"retail_silver.orders")

In [0]:
type(orders)

In [0]:
%sql
select min(order_date) as minDate, max(order_date) as maxDate from retail_silver.orders

In [0]:
orders.delete("order_date = '2013-07-25'")

In [0]:
%sql
select min(order_date) as minDate, max(order_date) as maxDate from retail_silver.orders

In [0]:
%sql
delete from retail_silver.orders where order_date = '2013-07-26'
    


In [0]:
%sql
select min(order_date) as minDate, max(order_date) as maxDate from retail_silver.orders

In [0]:
orders.update(condition= col("order_status")== "COMPLETE",
              set = {"order_status": lit("COMPLETED")})

In [0]:
%sql
select distinct order_status from retail_silver.orders

In [0]:
%sql
update retail_silver.orders set order_status = 'FRAUD' where order_status = "SUSPECTED_FRAUD"

# Merge Demo

In [0]:
%sql
select * from retail_test.departments

In [0]:
%sql
create or replace table retail_test.departments_tgr 
select * from retail_test.departments
    


In [0]:
%sql
select * from retail_test.departments

In [0]:
%sql
insert into retail_test.departments(id,name)
values(9,'Golf');

insert into retail_test.departments(id,name)
values(10,'Futniture');



In [0]:
%sql
Update retail_test.departments set name = 'Fan Fev' where id = 7;

In [0]:
%sql
DESCRIBE DETAIL retail_test.departments;


In [0]:
# from delta.tables import *
# from pyspark.sql.functions import lit,col
df_src = spark.sql("select * from retail_test.departments")
df_trgdlt = DeltaTable.forName(spark,"retail_test.departments_tgr")

df_trgdlt.alias("trg").merge(df_src.alias("src"),
                          "src.id = trg.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [0]:
%sql
select * from retail_test.departments_tgr

# Versioning

In [0]:
departments = DeltaTable.forName(spark, "retail_test.departments")
departments.history().display()

In [0]:
%sql
select * from retail_test.departments 
version as of 5

In [0]:
%sql
restore table retail_test.departments to version as of 2

# Delta Optimizatin _Techniques_

In [0]:
%sql
optimize retail_silver.orders

In [0]:
from delta.tables import *
from pyspark.sql.functions import lit,col
orders = DeltaTable.forName(spark,"retail_silver.orders")
orders.history().display()


In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")
orders.vacuum(150)

In [0]:
%sql
select * from retail_silver.orders version as of 2